In [1]:
import numpy as np
import cv2

In [2]:
with_mask = np.load('with_mask1.npy')
without_mask = np.load('without_mask1.npy')

In [4]:
with_mask = with_mask.reshape(with_mask.shape[0],100*100*3)
without_mask = without_mask.reshape(without_mask.shape[0],100*100*3)

In [5]:
X = np.r_[with_mask,without_mask]

In [6]:
labels = np.zeros(X.shape[0])

In [7]:
labels[with_mask.shape[0]:] = 1.0

In [8]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train , x_test ,y_train , y_test = train_test_split(X,labels,test_size = 0.05)

In [13]:
svm = SVC(probability = True)
svm.fit(x_train,y_train)
# print(y_train)

SVC(probability=True)

In [14]:
y_pred = svm.predict(x_test)

In [15]:
accuracy_score(y_test,y_pred)

0.9114583333333334

In [150]:
category = ['Mask','No Mask']
haar_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
data = []
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag,img = cap.read()
    if flag:
        rect_arr = haar_data.detectMultiScale(img)        
        
        for x,y,w,h in rect_arr:
            cv2.rectangle(img,(x,y),(x+w,y+w),(255,0,255),4)
            face = img[y:y+h,x:x+w,:]
            face = cv2.resize(face,(100,100))
            face = face.reshape(1,-1)
#             face = pca.transform(face)
            pred = svm.predict(face)
            n = category[int(pred)]
            cv2.putText(img,n,(x,y),font,1,(230,230,230),2)

        cv2.imshow('result',img)
        if cv2.waitKey(2) == 27:
            break
cap.release()
cv2.destroyAllWindows()
    

In [17]:
import pickle
with open('ml_module.pkl','wb') as fp:
    pickle.dump(svm,fp)